## Import Dependencies

In [2]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
#spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:15 http://se

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-01-06 00:15:41--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.12MB/s    in 0.8s    

2022-01-06 00:15:43 (1.12 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



## Start Session and Import Data

In [4]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("VideoGames").getOrCreate()

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz'
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True)

# show dataframe
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

## Data Processing

In [6]:
# count number of records (rows) in dataset
print(f"The number of records in the video game reviews dataset is: {df.count()}")

The number of records in the video game reviews dataset is: 1785997


In [15]:
# drop rows with na values
df = df.dropna()

# drop duplicate rows
df = df.dropDuplicates()

# count new number of records
print(f"The number of records in the video game reviews dataset  after cleaning is: {df.count()}")

The number of records in the video game reviews dataset  after cleaning is: 1785886


In [16]:
# pull out fields for review_id_table
review_id_table = df['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date']

# view column dtypes to ensure they match the sql schema
review_id_table.dtypes

[('review_id', 'string'),
 ('customer_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'string'),
 ('review_date', 'string')]

In [17]:
# change dtype of customer_id and product_parent to integer and review_date to date
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DateType
review_id_table = review_id_table.withColumn('customer_id', review_id_table['customer_id'].cast(IntegerType()))
review_id_table = review_id_table.withColumn('product_parent', review_id_table['product_parent'].cast(IntegerType()))
review_id_table = review_id_table.withColumn('review_date', review_id_table['review_date'].cast(DateType()))

# confirm dtypes have been fixed
review_id_table.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [18]:
# pull out fields for products table
products = df['product_id', 'product_title']

# view column dtypes to ensure they match the sql schema
products.dtypes

[('product_id', 'string'), ('product_title', 'string')]

Both columns of this table have the correct datatype

In [19]:
# count number of reviews for each customer
customers = df.groupBy('customer_id').count()

# rename count column to customer_count and change dtypes to int to match sql schema
customers = customers.withColumn('customer_id', customers['customer_id'].cast(IntegerType()))
customers = customers.withColumnRenamed('count', 'customer_count')
customers = customers.withColumn('customer_count', customers['customer_count'].cast(IntegerType()))

# show table
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    5401088|             5|
|   37964777|            16|
|   20872523|             1|
|   44777937|             4|
|    2384511|             1|
|   41500251|             1|
|   34029693|             1|
|   40005173|             2|
|   12125016|             2|
|   21226328|             3|
|    6004768|             2|
|   14445274|             4|
|   41665035|             1|
|   16995171|             1|
|   25044130|             1|
|   11073960|             2|
|   38642776|             8|
|    6199838|             6|
|   19207466|             3|
|    1760318|             1|
+-----------+--------------+
only showing top 20 rows



In [20]:
customers.dtypes

[('customer_id', 'int'), ('customer_count', 'int')]

In [21]:
# pull out fields for vine_table
vine_table = df['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine']

# check column dtypes
vine_table.dtypes

[('review_id', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'string'),
 ('total_votes', 'string'),
 ('vine', 'string')]

In [22]:
# cast the star_rating, helpful_votes, and total_votes columns as integers
vine_table = vine_table.withColumn('star_rating', vine_table['star_rating'].cast(IntegerType()))
vine_table = vine_table.withColumn('helpful_votes', vine_table['helpful_votes'].cast(IntegerType()))
vine_table = vine_table.withColumn('total_votes', vine_table['total_votes'].cast(IntegerType()))

# recheck column dtypes to ensure they have changed to fit the schema
vine_table.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string')]